# Ver. 1

In [ ]:
import os

from langchain.schema.runnable import RunnableMap
from langchain.prompts import ChatPromptTemplate
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv


In [ ]:
load_dotenv()


In [ ]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)

In [ ]:
with open(r'assets/general_qna - Copy.txt','r') as file:
    data = file.read()


In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectorstore = DocArrayInMemorySearch.from_texts(
    # mini docs for embedding
    data.split('\n'),
    embedding=embeddings # passing in the embedder model
)


In [ ]:
template = """
Your name is SAMBHU who is an ARTIFICIAL INTELLIGENCE who help Students to remain query free
Answer the question in humanized sentence, based only on the following context:

WHEN question is out of context say the below: 
`Can't help here you may contact here :https://wa.me/+919819342606` except if you are asked about you identity

{context}

Return Answer in single sencentence in markdown format with futher which can be asked

Question: {question}
"""


In [ ]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()


retriever = vectorstore.as_retriever()

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | output_parser

In [ ]:
chain

In [ ]:
print(chain.invoke({"question": "Need farewell registeration link and dates"}))

In [ ]:
retriever.get_relevant_documents("do we need to register for farewell  ? ")

# ver. 2

In [45]:
import os
import pandas as pd
import google.generativeai as genai
from uuid import uuid4
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from datetime import datetime
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_google_genai import (ChatGoogleGenerativeAI,
                                    GoogleGenerativeAIEmbeddings)
from database import connect_db
import psycopg2


In [46]:
load_dotenv()


True

In [68]:
template = """
[INTRODUCTION]
Your name is KHALNAIK, You are an ARTIFICIAL INTELLIGENCE who help Students to remain query free 
You were Created and owned by someone known Tony Stank...😼 maybe, 

[GREETINGS]
On receiving any greetings make sure to greet back

[HELP]
On prompt of `Help` suggest some questions the user could ask you !

[OUT OF CONTEXT ANSWERS]
Answer the question in humanized sentence, based following context:

WHEN question is out of context say the below with the reason for not answering: 
```
Can't help here you may use the followings below :
- [WhatsApp](https://wa.me/+919819342606)
- [RSET WEBSITE](https://www.rset.edu.in/gscc)```

[CONTEXT]

<context>
{context}
</context>

[SUGGESTED QUESTIONS]
Return Answer in single sencentence in markdown format

Suggest more questions with in the context in below format [OPTIONAL]:
**You can also ask for** 
 - 
 - 
 - 

Question:{input}
"""


In [83]:
# Configure Google Generative AI
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


# Initialize Google Generative AI chatbot
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

with open(r'assets/general_qna - Copy.txt', 'r') as file:
    data = file.read()

# Initialize embeddings and vectorstore
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Text splitting component
text_splitter = RecursiveCharacterTextSplitter()  # Clearly instantiate the splitter
splits = text_splitter.split_text(data)  # Split the input data

# Vector store for embedding representations
vectorstore = FAISS.from_texts(splits, embeddings)  # Construct vector store from text and embeddings
retriever = vectorstore.as_retriever()  # Designate the vector store as a retrieval engine 

# Document prompt creation
doc_prompt = ChatPromptTemplate.from_template(template=template)  # Create prompt from template
doc_chain = doc_prompt | llm | StrOutputParser()  # Chain prompt, LLM, and output parser

# Retriever integration
retriever_chain = create_retrieval_chain(retriever, doc_chain)  # Build chain for retr


In [90]:
response = retriever_chain.invoke({
            'input': 'Give me notes for ITSM'+'also make sure to share me the most relevant link'
        })
print(response)

{'input': 'Give me notes for ITSMalso make sure to share me the most relevant link', 'context': [Document(page_content='[Semester 5] - full Syllabus at https://drive.google.com/file/d/1G6E_Sy5Jz679oUqh8oVay8rzhXM1p26H/view?usp=drivesdk\nSemester 5 notes for all subjects available https://drive.google.com/drive/u/0/folders/1z8LI-QtaaWDLpz2tzSUetW4sJvmNOblp\nSemester 5 notes for Enterprise Java (EJ) available https://drive.google.com/drive/u/0/folders/1mcHyH5EawLjyM83jDNFSmnOYYGSm9PA1\nSemester 5 notes for Artificial Intelligence (AI) available https://drive.google.com/drive/folders/1vaXpGYOs6ij2yNhVKia0fvKE3uVCL58Q\nSemester 5 notes for Advanced Web Programming (AWP) available https://drive.google.com/drive/u/0/folders/1vZjcqGiBGkauifd52gHXydPKe16Jw9Ak\nSemester 5 notes for Internet Of Things (IOT) available https://drive.google.com/drive/u/0/folders/121nk9A6nlDAqB27LCT6pugyaL1aqmT6s\nSemester 5 notes for Software Project Management (SPM) available https://drive.google.com/drive/u/0/fol

In [91]:
print(response['answer'])

Here is the link for Semester 6 notes for Information Technologies Service Management (ITSM) : https://drive.google.com/drive/folders/1vaXpGYOs6ij2yNhVKia0fvKE3uVCL58Q

**You can also ask for** 
 - Syllabus for Semester 6
 - Notes for other subjects in Semester 6
 - Deadlines for submission in Semester 6


In [93]:

with open(r'assets/general_qna.txt', 'r') as file:
    data = file.read()

with open(r'assets/class_qna.txt', 'r') as file:
    data += file.read()



In [94]:
data

"Rajasthani Sammelan Education Trust (RSET) Overview:\n\n- Established in 1948 at Malad, Mumbai\n- Registered public charitable trust\n- Aims to promote educational, charitable, and social activities\n- Provides educational opportunities from primary to post-graduation\n- Fields include Commerce, Management, Information Technology, and Media Studies\n- Caters to social and humanitarian needs\n- Initially started educational activities for girls, later opened to boys\n- Over 15,000 students currently enrolled in various institutions\n- Thousands more enrolled with Kunjbihari S. Goyal Online Academy (GOLD)\n\nEducational Institutions and Organizations:\n\nEducational Institutions owned and run by RSET:\n- Kudilal Govindram Seksaria Sarvodaya School\n- Kudilal Govindram Seksaria English School\n- Ramniwas Bajaj English High School\n- Mainadevi Bajaj International School\n- Mainadevi Bajaj International Playschool\n- Durgadevi Saraf Junior College\n- Ghanshyamdas Saraf College of Arts & Co